# Transformers 模型量化技术：GPTQ

In [1]:
# 或者一次性列出所有已安装包及其版本（可搜索关键词）
!pip list | grep -E "torch|transformers|accelerate|bitsandbytes|datasets|optimum"

/bin/bash: /root/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
accelerate                0.26.1
datasets                  4.0.0
optimum                   1.27.0
torch                     2.7.1
torchaudio                2.7.1
torchcodec                0.5
torchvision               0.22.1
transformers              4.41.1


In [2]:
!nvidia-smi

/bin/bash: /root/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Sat Aug  2 22:21:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        On  |   00000000:0B:00.0  On |                  N/A |
| 36%   46C    P8             36W /  320W |    1920MiB /  10240MiB |     13%      Default |
|                                   

In [3]:
import torch
print(torch.cuda.is_available())  # 应输出True
print(torch.version.cuda)  # 应与nvcc -V显示的版本一致（或兼容）

True
12.6


In [4]:
import torch
print(torch.cuda.is_available())  # 应输出True
print(torch.version.cuda)  # 应与nvcc -V显示的版本一致（或兼容）

True
12.6


In [5]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-1.3b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

In [7]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

/root/miniconda3/envs/aistudy5/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/root/miniconda3/envs/aistudy5/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/root/miniconda3/envs/aistudy5/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


Quantizing model.decoder.layers blocks :   0%|          | 0/24 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/root/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [8]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': {},
 '_buffers': {'qweight': tensor([[-1968072024, -1771280518,  1789289879,  ...,  1215579015,
            2053435976, -1753460124],
          [  967351451, -1785370714, -1778418564,  ...,  -878163915,
            -110706971,  2105174792],
          [  844485784, -1236563623,  -882554554,  ..., -1999525490,
           -1734957961,  2020046922],
          ...,
          [ 1821035160, -1280943212, -1672110441,  ..., -1186514519,
            1723308183, -1263167625],
          [-1516574247, -1471756296, -1236731976,  ..., -2004788788,
            1468696729,  1719643110],
          [ -942189433, -1148750501, -1505130619,  ..., -1702438745,
             -60205431,  -912684712]], device='cuda:0', dtype=torch.int32),
  'qzeros': tensor([[2004318071, 2004318071, 2004318071,  ..., 2004318071, 2004318071,
           2004318071],
          [2004318071, 2004318071, 2004318071,  ..., 2004318071, 2004318071,
           2004318071],
          [2004318071, 20043180

In [9]:
quant_model.save_pretrained("models/opt-1.3b-gptq")

#### 使用 GPU 加载模型并生成文本

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still around. I'm not sure if you're still around, but I'd love to see you again.
I'm still around, but I'm not sure if I'll be around for the next few days. I'll try to make it to the next one.
I'm glad to hear


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

In [11]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-1.3b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

Quantizing model.decoder.layers blocks :   0%|          | 0/24 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/root/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [12]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to be.



 is a
 to be.

 is a to be.
 is a to.
 is a to. is a to. is a to. is a to. is a to. is a to. is a to. is a to. is a to. is a to


## [Optional]Homework: 使用 GPTQ 算法量化 OPT-6.7B 模型